In [ ]:
!git clone https://github.com/mohitksahu/agentic-ai.git

Cloning into 'agentic-ai'...


In [ ]:
cd agentic-ai

In [ ]:
!pip install -r requirements.txt

# 💰 Financial Analysis System

A streamlined financial analysis tool that processes your CSV data and provides intelligent insights.

## 🎯 Core Features:
1. **CSV Data Processing** - Upload and analyze your financial CSV files
2. **Budget Analysis** - Comprehensive spending breakdown and savings calculation
3. **RAG-based Q&A** - Ask questions about your financial data and get intelligent answers

## 📊 How It Works:
1. Upload your financial CSV files with transaction data
2. Get detailed budget analysis and spending patterns
3. Ask specific questions about your finances using the RAG model

## 📁 CSV Format Requirements:
- **Date column**: Transaction dates (auto-detected)
- **Amount column**: Transaction amounts
- **Category/Description column**: Spending categories or descriptions

> **Privacy First**: All analysis happens locally - your data never leaves your computer.

# 🚀 Quick Start Guide

## Workflow:
1. **Setup Environment** - Install dependencies and initialize components
2. **Upload CSV Data** - Add your financial transaction CSV files
3. **Generate Analysis** - Get comprehensive budget analysis and insights
4. **Ask Questions** - Use the RAG model to query your financial data

## ⚡ Quick Setup:
Run the cells in order to get started with your financial analysis.

In [ ]:
# 1. Environment Setup
try:
    from utils.environment_setup import EnvironmentSetup

    env_setup = EnvironmentSetup()

    # Setup directories
    dir_result = env_setup.setup_directories()
    if dir_result['success']:
        paths = dir_result['paths']
        print(f"📁 Working directory: {paths['current']}")

        if dir_result['created']:
            print(f"✅ Created directories: {', '.join(dir_result['created'])}")
        if dir_result['existing']:
            print(f"✓ Existing directories: {', '.join(dir_result['existing'])}")

        # Store paths globally for other cells
        current_dir = paths['current']
        input_dir = paths['input']
        output_dir = paths['output']

        print(f"📁 Data directories ready:")
        print(f"   📥 Input: {input_dir}")
        print(f"   📤 Output: {output_dir}")
    else:
        print(f"❌ Directory setup failed: {dir_result['error']}")

    # Setup Python path
    path_result = env_setup.setup_python_path()
    if path_result['success']:
        if path_result['added']:
            print("✅ Added project to Python path")
        else:
            print("✓ Project already in Python path")
    else:
        print(f"⚠️ Python path setup issue: {path_result['error']}")

except Exception as e:
    print(f"❌ Environment setup failed: {e}")
    print("💡 Using fallback setup...")
    # Fallback setup
    from pathlib import Path
    import sys
    current_dir = Path.cwd()
    input_dir = current_dir / "data" / "input"
    output_dir = current_dir / "data" / "output"
    input_dir.mkdir(parents=True, exist_ok=True)
    output_dir.mkdir(parents=True, exist_ok=True)
    if str(current_dir) not in sys.path:
        sys.path.append(str(current_dir))
    print("✅ Fallback environment setup complete")

In [ ]:
# 2. Install Dependencies
try:
    from utils.dependency_manager import DependencyManager

    dep_manager = DependencyManager()

    print("📦 Installing required packages...")
    result = dep_manager.install_packages()

    if result['success']:
        print("✅ Dependencies installed successfully!")
    else:
        print(f"⚠️ Installation issues: {result.get('stderr', 'Unknown error')}")
        # Check what's available
        missing = dep_manager.get_missing_packages()
        if missing:
            print(f"📋 Missing packages: {', '.join(missing)}")

except Exception as e:
    print(f"⚠️ Dependency manager failed: {e}")
    print("💡 Using direct pip install...")
    import subprocess
    import sys
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install',
                             'pandas', 'numpy', 'matplotlib', 'seaborn',
                             'plotly', 'transformers', 'torch', '-q'])
        print("✅ Fallback installation complete!")
    except Exception as fallback_error:
        print(f"❌ Fallback installation failed: {fallback_error}")

In [ ]:
# 3. Initialize Core Components
print("🤖 Loading components...")

# Budget calculator for real financial data
class BudgetCalculator:
    def analyze_spending(self, df):
        """Analyze spending from real CSV data"""
        try:
            if len(df) == 0:
                return {
                    'total_spending': 0,
                    'total_categories': 0,
                    'category_breakdown': {},
                    'recommendations': ['No transaction data available - please load CSV files']
                }

            # Find amount column in user's data
            amount_col = None
            for col in df.columns:
                if any(keyword in col.lower() for keyword in ['amount', 'value', 'cost', 'price']):
                    amount_col = col
                    break

            if amount_col is None:
                amount_col = df.columns[1] if len(df.columns) > 1 else df.columns[0]

            total_spending = abs(df[amount_col].sum())

            # Find category column in user's data
            category_col = None
            for col in df.columns:
                if any(keyword in col.lower() for keyword in ['category', 'description', 'type', 'desc']):
                    category_col = col
                    break

            if category_col and category_col in df.columns:
                categories = df.groupby(category_col)[amount_col].sum().abs().to_dict()
                total_categories = len(categories)

                # Generate insights based on actual data
                top_category = max(categories, key=categories.get)
                top_amount = categories[top_category]

                recommendations = [
                    f"Total spending across {total_categories} categories: ${total_spending:.2f}",
                    f"Highest spending category: {top_category} (${top_amount:.2f})",
                    f"Average per category: ${total_spending/total_categories:.2f}"
                ]
            else:
                categories = {'Uncategorized': total_spending}
                total_categories = 1
                recommendations = [
                    "Add category column to CSV for detailed breakdown",
                    f"Total spending: ${total_spending:.2f}"
                ]

            return {
                'total_spending': total_spending,
                'total_categories': total_categories,
                'category_breakdown': categories,
                'recommendations': recommendations
            }

        except Exception as e:
            return {
                'total_spending': 0,
                'total_categories': 0,
                'category_breakdown': {},
                'recommendations': [f'Error analyzing data: {str(e)}']
            }

# Financial Q&A agent for real data queries
class FinancialAgent:
    def ask_question(self, question, data_context):
        """Answer questions about user's actual financial data"""
        try:
            q_lower = question.lower()

            if 'total' in q_lower and any(word in q_lower for word in ['spend', 'expense']):
                total = data_context.get('total_expenses', data_context.get('total_spending', 0))
                return f"💰 Your total spending is ${total:.2f} based on your CSV data"

            elif any(word in q_lower for word in ['highest', 'most', 'top']) and 'category' in q_lower:
                categories = data_context.get('category_breakdown', {})
                if categories:
                    top_cat = max(categories, key=categories.get)
                    top_amount = categories[top_cat]
                    return f"? Your highest spending category is {top_cat} with ${top_amount:.2f}"
                return "📊 No category data found in your CSV"

            elif any(word in q_lower for word in ['average', 'avg']):
                avg = data_context.get('avg_transaction', 0)
                return f"📈 Your average transaction amount is ${avg:.2f}"

            elif 'categor' in q_lower and any(word in q_lower for word in ['how many', 'number']):
                count = data_context.get('total_categories', 0)
                return f"? You have {count} spending categories in your data"

            elif any(word in q_lower for word in ['recommend', 'advice', 'suggest']):
                recommendations = data_context.get('recommendations', [])
                if recommendations:
                    return "💡 Based on your data: " + "; ".join(recommendations[:2])
                return "? Load your CSV data to get personalized recommendations"

            else:
                return "🤖 I can answer questions about your total spending, categories, averages, and provide recommendations based on your CSV data"

        except Exception as e:
            return f"❌ Error processing question: {str(e)}"

# Initialize components
budget_calc = BudgetCalculator()
financial_agent = FinancialAgent()

print("✅ Components initialized:")
print("   📊 Budget Calculator - Ready for real data")
print("   🤖 Financial Agent - Ready for real data Q&A")

# 4. 📁 CSV Data Input

Upload your financial CSV files to get started with analysis.

## Required CSV Structure:
- **Date column**: Transaction dates (any common date format)
- **Amount column**: Transaction amounts (positive or negative numbers)  
- **Category/Description**: Spending categories or transaction descriptions

## Supported Column Names:
- **Date**: `date`, `transaction_date`, `Date`, `DATE`
- **Amount**: `amount`, `Amount`, `value`, `price`, `cost`
- **Category**: `category`, `description`, `type`, `desc`

> **Tip**: The system automatically detects your column names, so don't worry about exact formatting!

In [ ]:
# 4. CSV Data Processing
print("📊 Processing CSV data...")

try:
    import pandas as pd

    # Find CSV files in input directory
    csv_files = [f for f in input_dir.glob("*.csv") if f.stat().st_size > 0]

    if csv_files:
        # Load first available CSV file
        csv_file = csv_files[0]

        print(f"📁 Loading: {csv_file.name}")

        # Load CSV data
        combined_financial_data = pd.read_csv(csv_file)

        print(f"✅ Successfully loaded {len(combined_financial_data)} transactions")
        print(f"📋 Columns: {list(combined_financial_data.columns)}")

        # Display data preview
        try:
            from IPython.display import display
            print("\n📋 Data Preview:")
            display(combined_financial_data.head())
        except:
            print("\n📋 Data Preview:")
            print(combined_financial_data.head().to_string())

    else:
        print("❌ No CSV files found in input directory!")
        print(f"📁 Please add your financial CSV files to: {input_dir}")
        print("💡 Required format: CSV with columns for date, amount, and category/description")

        # Set empty dataframe to prevent errors in subsequent cells
        combined_financial_data = pd.DataFrame()

except Exception as e:
    print(f"❌ Error processing CSV data: {e}")
    print(f"? Please ensure CSV files are properly formatted in: {input_dir}")

    # Set empty dataframe to prevent errors
    import pandas as pd
    combined_financial_data = pd.DataFrame()

print("📊 CSV data processing complete")

# 📊 Data Visualization - Transaction Analysis

Generate comprehensive visualizations from your processed CSV data:

- **Transaction Trends**: Timeline of your spending patterns
- **Category Analysis**: Breakdown of expenses by category
- **Spending Distribution**: Analysis of transaction amounts and frequency
- **Time Patterns**: Weekly and monthly spending behaviors

These visualizations will help the RAG model provide more detailed insights about your financial habits.

In [ ]:
# 5. Budget Analysis and Financial Insights
print("📊 Generating budget analysis...")

try:
    if 'combined_financial_data' in locals() and not combined_financial_data.empty:

        # Budget calculation using actual data
        if 'budget_calc' in locals():
            try:
                # Use actual data for analysis - no demo calculations
                total = combined_financial_data['amount'].sum()
                avg_transaction = combined_financial_data['amount'].mean()

                print(f"\n💰 Financial Analysis:")
                print(f"Total Spending: ${abs(total):.2f}")
                print(f"Average Transaction: ${abs(avg_transaction):.2f}")
                print(f"Transaction Count: {len(combined_financial_data)}")

                if 'category' in combined_financial_data.columns:
                    print("\n📊 Category Breakdown:")
                    category_summary = combined_financial_data.groupby('category')['amount'].agg(['sum', 'count']).round(2)
                    for category in category_summary.index:
                        amount = abs(category_summary.loc[category, 'sum'])
                        count = category_summary.loc[category, 'count']
                        print(f"  • {category}: ${amount:.2f} ({count} transactions)")

                # Generate recommendations based on actual spending patterns
                if 'category' in combined_financial_data.columns:
                    top_category = combined_financial_data.groupby('category')['amount'].sum().abs().idxmax()
                    top_amount = abs(combined_financial_data.groupby('category')['amount'].sum()[top_category])

                    print(f"\n💡 Insights:")
                    print(f"  • Highest spending category: {top_category} (${top_amount:.2f})")
                    print(f"  • Number of spending categories: {combined_financial_data['category'].nunique()}")

            except Exception as e:
                print(f"⚠️ Budget calculation error: {e}")
                print("Please ensure your CSV has proper amount and category columns")
        else:
            print("⚠️ Budget calculator not available")
            print("Please run the component initialization cell first")

    else:
        print("❌ No financial data available for analysis")
        print("? Please add CSV files to the input directory and run the data processing cell")

except Exception as e:
    print(f"❌ Analysis error: {e}")
    print("Please ensure CSV data is properly loaded before running analysis")

print("📊 Budget analysis complete")

# 5. 📊 Budget Analysis

Generate comprehensive financial analysis from your CSV data:

- **Spending Breakdown**: Analyze expenses by category
- **Savings Calculation**: Calculate your savings rate and budget surplus/deficit  
- **Budget Recommendations**: Get personalized advice based on your spending patterns
- **Financial Health Check**: Assess your overall financial wellness

In [ ]:
# 6. Data Visualization Generation
print("📊 Generating visualizations...")

try:
    if 'combined_financial_data' in locals() and not combined_financial_data.empty:

        print("📈 Creating financial charts...")

        # Check if visualization packages are available
        try:
            import matplotlib.pyplot as plt
            import seaborn as sns

            # Set visualization style
            plt.style.use('default')

            if 'category' in combined_financial_data.columns and 'amount' in combined_financial_data.columns:
                # Category spending chart
                plt.figure(figsize=(10, 6))

                category_totals = combined_financial_data.groupby('category')['amount'].sum().sort_values(ascending=False)

                # Bar chart
                plt.subplot(1, 2, 1)
                category_totals.plot(kind='bar', color='skyblue')
                plt.title('Spending by Category')
                plt.xlabel('Category')
                plt.ylabel('Amount ($)')
                plt.xticks(rotation=45)

                # Pie chart
                plt.subplot(1, 2, 2)
                category_totals.plot(kind='pie', autopct='%1.1f%%', startangle=90)
                plt.title('Spending Distribution')
                plt.ylabel('')

                plt.tight_layout()

                # Save to output directory
                output_vis_dir = output_dir / "visualizations"
                output_vis_dir.mkdir(exist_ok=True)

                chart_path = output_vis_dir / "spending_analysis.png"
                plt.savefig(chart_path, dpi=300, bbox_inches='tight')

                print(f"? Chart saved: {chart_path}")
                plt.show()

            else:
                print("⚠️ Required columns (category, amount) not found in data")
                print("📝 Available columns:", list(combined_financial_data.columns))

        except ImportError:
            print("📊 Visualization libraries not available")
            print("💡 Install with: pip install matplotlib seaborn")
            print("📄 Generating text-based visualization...")

            # Text-based visualization
            if 'category' in combined_financial_data.columns:
                print("\n📊 Text-Based Category Summary:")
                category_totals = combined_financial_data.groupby('category')['amount'].sum().sort_values(ascending=False)

                max_amount = category_totals.max()
                for category, amount in category_totals.items():
                    bar_length = int((amount / max_amount) * 30)  # Scale to 30 characters
                    bar = "█" * bar_length + "░" * (30 - bar_length)
                    print(f"{category:15} │{bar}│ ${amount:.2f}")

        except Exception as viz_error:
            print(f"⚠️ Visualization error: {viz_error}")

    else:
        print("⚠️ No data available for visualization")
        print("? Sample chart would show spending patterns when data is loaded")

except Exception as e:
    print(f"❌ Visualization error: {e}")
    print("📊 Charts will be generated when data is properly loaded")

print("📊 Visualization generation complete")

# 💰 Budget & Savings Visualization

Generate advanced financial analysis visualizations with budget insights:

- **Income vs Expenses**: Visual comparison of income and spending
- **Savings Rate Analysis**: Progress tracking and savings goals
- **Budget Allocation**: How your income is distributed across categories
- **Financial Health Metrics**: Key performance indicators for your finances

These visualizations provide comprehensive insights for the RAG model to analyze your financial health and provide personalized recommendations.

In [ ]:
# 7. Financial Q&A Chat Interface
print("💬 Financial Q&A system ready")

try:
    if 'financial_agent' in locals():

        print("\n🤖 AI Financial Assistant")
        print("Ask me anything about your financial data!")
        print("Example questions:")
        print("• What's my highest expense category?")
        print("• How much did I spend on groceries?")
        print("• What are some budget recommendations?")

        if 'combined_financial_data' in locals() and not combined_financial_data.empty:
            print("\n✅ Q&A system ready with your data")
            print("💬 Interactive Q&A: Use financial_agent.ask_question(your_question, context)")

            # Create context from actual data for Q&A
            if 'category' in combined_financial_data.columns and 'amount' in combined_financial_data.columns:
                total_expenses = abs(combined_financial_data['amount'].sum())
                categories = combined_financial_data.groupby('category')['amount'].sum().abs().to_dict()

                context = {
                    'total_expenses': total_expenses,
                    'category_breakdown': categories,
                    'transaction_count': len(combined_financial_data),
                    'avg_transaction': abs(combined_financial_data['amount'].mean())
                }

                print(f"📊 Data loaded: {len(combined_financial_data)} transactions, ${total_expenses:.2f} total")
            else:
                print("⚠️ CSV data needs 'category' and 'amount' columns for full Q&A functionality")
        else:
            print("❌ No financial data loaded")
            print("? Please add CSV files to input directory and run data processing first")

    else:
        print("⚠️ Financial agent not initialized")
        print("💡 Please run the component initialization cell first")

except Exception as e:
    print(f"❌ Q&A system error: {e}")
    print("Ensure data is loaded and components are initialized")

print("💬 Q&A system initialization complete")

# 6. 🤖 RAG-Based Financial Q&A

Ask questions about your financial data and get intelligent, context-aware answers.

## Example Questions:
- *"How much am I saving each month?"*
- *"What's my biggest spending category?"*
- *"Is my savings rate healthy?"*
- *"How can I improve my budget?"*
- *"What percentage of income goes to housing?"*

The RAG model will analyze your actual financial data to provide personalized answers.

In [ ]:
# 8. System Summary and Status
print("📋 Agentic AI Financial System - Status Summary")
print("=" * 50)

try:
    # Environment status
    print("🔧 Environment Status:")
    print(f"   Working Directory: {Path.cwd()}")
    print(f"   Input Directory: {'✅ Available' if input_dir.exists() else '❌ Missing'}")
    print(f"   Output Directory: {'✅ Available' if output_dir.exists() else '❌ Missing'}")

    # Check for CSV files in input
    csv_files = list(input_dir.glob("*.csv")) if input_dir.exists() else []
    print(f"   CSV Files Found: {len(csv_files)}")

    # Component status
    print("\n🧩 Component Status:")
    components_status = {
        'Budget Calculator': 'budget_calc' in locals(),
        'Financial Agent': 'financial_agent' in locals(),
        'CSV Data Loaded': 'combined_financial_data' in locals() and not combined_financial_data.empty if 'combined_financial_data' in locals() else False
    }

    for component, status in components_status.items():
        status_icon = "✅" if status else "❌"
        print(f"   {status_icon} {component}")

    # Data summary - only show if real data exists
    print("\n📊 Data Summary:")
    if 'combined_financial_data' in locals() and not combined_financial_data.empty:
        print(f"   Records Loaded: {len(combined_financial_data)}")
        print(f"   Columns: {', '.join(combined_financial_data.columns)}")

        if 'amount' in combined_financial_data.columns:
            total_amount = abs(combined_financial_data['amount'].sum())
            print(f"   Total Amount: ${total_amount:.2f}")

        if 'category' in combined_financial_data.columns:
            categories = combined_financial_data['category'].nunique()
            print(f"   Categories: {categories}")
    else:
        print("   ❌ No financial data loaded")
        print("   💡 Add CSV files to data/input directory")

    # Features status
    print("\n🚀 Available Features:")
    data_loaded = 'combined_financial_data' in locals() and not combined_financial_data.empty if 'combined_financial_data' in locals() else False

    features = [
        ("📊 Budget Analysis & Insights", data_loaded),
        ("📈 Data Visualization", data_loaded),
        ("💬 AI-Powered Q&A", data_loaded and 'financial_agent' in locals()),
        ("📁 CSV Data Processing", True),
        ("📋 Financial Reporting", data_loaded)
    ]

    for feature, available in features:
        icon = "✅" if available else "⏳"
        status = "Ready" if available else "Requires CSV data"
        print(f"   {icon} {feature} - {status}")

    if data_loaded:
        print("\n✅ System Ready for Financial Analysis")
    else:
        print("\n⏳ System Ready - Add CSV data to begin analysis")
        print("💡 Place your financial CSV files in data/input/ directory")

except Exception as e:
    print(f"❌ System status error: {e}")
    print("🔧 Some components may need reinitialization")

print("\n" + "=" * 50)
if 'combined_financial_data' in locals() and not combined_financial_data.empty:
    print("🎯 Ready to Analyze Your Financial Data!")
else:
    print("🎯 Ready for CSV Data Upload - Add Files to Begin!")